In [2]:
from gensim.models import word2vec
import gensim.models
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt
from numba import jit# 加速套件
import glob
import os
import statistics as stat
from scipy import stats
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import xlsxwriter
import numpy as np
import numpy as ndarray
from gensim.models import word2vec
#from gensim.models import Word2vec
from gensim import models
import logging
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
import jieba
from gensim.test.utils import datapath

# 以下是順序

In [3]:
"""
1.載入檔案,這個檔案是每篇文章的內文與平均向量,拿出平均向量那欄,存到articles_matrix裡面
2.載入bin檔,這個檔案是訓練好的bin檔,裡面有每個詞的向量
測試開始
3.輸入文字:def 請輸入文字(input_words)把輸入的文章斷詞成字
4.獲取輸入詞的平均向量:def get_article_avgvector(wordlist):把斷完詞的單字利用先前載入的bin檔依序變成向量,加總後再取平均
於是現在有了輸入的文章平均詞向量,接著,只要與原本的文章向量做比對,就可以找到最相近的文章
5.餘弦相似度:def cos_sim(vector_a, vector_b):這個函數把輸入的向量跟5000篇新聞逐一比對餘弦相似度
6.比對:def 餘閒相似度找文章(input_words):這個函數把5.比對的結果存起來,再找出最相近的一篇列出來
"""

'\n1.載入檔案,這個檔案是每篇文章的內文與平均向量,拿出平均向量那欄,存到articles_matrix裡面\n2.載入bin檔,這個檔案是訓練好的bin檔,裡面有每個詞的向量\n測試開始\n3.輸入文字:def 請輸入文字(input_words)把輸入的文章斷詞成字\n4.獲取輸入詞的平均向量:def get_article_avgvector(wordlist):把斷完詞的單字利用先前載入的bin檔依序變成向量,加總後再取平均\n於是現在有了輸入的文章平均詞向量,接著,只要與原本的文章向量做比對,就可以找到最相近的文章\n5.餘弦相似度:def cos_sim(vector_a, vector_b):這個函數把輸入的向量跟5000篇新聞逐一比對餘弦相似度\n6.比對:def 餘閒相似度找文章(input_words):這個函數把5.比對的結果存起來,再找出最相近的一篇列出來\n'

In [14]:
#1.載入檔案
article = pd.read_excel(r"C:\Users\Big data\Desktop\class\funcardproject\data\文章\新聞\article_news_vector _final.xlsx")
articles_matrix =[get_article_matrix(article,i) for i in range(5000)]

In [16]:
article

,No.,content,article_vector_matrix
0,0,國內刷卡回饋重洗牌 無腦刷排行榜大車拼\n\n\n 下半年信用卡市場「流血搶市」，多家銀行...,"0.0002087556931655854, -0.0005011226749047637,..."
1,1,直擊雙11電商AI倉儲 下單到出貨僅10分鐘\n\n\n 雙11狂熱訂單爆量，電商平台拼「...,"-0.00029341760091483593, -0.000209834950510412..."
2,2,雙11網購主打賓士車 交易攻略回饋16.8萬元\n\n\n 雙11購物節戰火激烈，網購平台...,"0.0008746232488192618, -0.0006640242645516992,..."
3,3,刷信用卡繳納地價稅 24家銀行免收手續費\n\n\n 今(2019)年最後一項稅款地價稅，...,"0.0002632589603308588, -0.0006318831001408398,..."
4,4,4張行動支付聯名卡對決 街口卡力尬Pi拍錢包\n\n\n 隨著街口聯名卡上市，炒熱行動支付...,"0.0002618952130433172, -0.000628599664196372, ..."
5,5,零售4大錢包各顯神威 支付累點還能存零錢\n\n\n 在行動支付的浪潮下，傳統的零售通路展...,"0.00026534072821959853, -0.0006368649774231017..."
6,6,前8月刷卡量衝2兆元 玉山包辦第3季發卡王\n\n\n 歷經6月報稅入帳的高峰後，今(20...,"0.00012652100122068077, -0.0003037579590454697..."
7,7,刷信用卡如打怪衝等 累積紅利刺激養卡潮\n\n\n 信用卡市場戰火連天，持卡人「逐回饋而刷...,"-0.00015988782979547977, -0.000830913020763546..."
8,8,電信聯名卡升級激戰 購機回饋分期0利率\n\n\n iPhone新機上市引爆電信續約與換約...,"0.00023613832308910787, -0.000566807808354497,..."
9,9,全聯多元支付大解放 實體信用卡封印解除\n\n\n 全聯福利中心強力推出「PX Pay」行...,"0.00022709710174240172, -0.0005451262113638222..."


In [12]:
#2.載入bin檔
wv_from_bin = KeyedVectors.load_word2vec_format(datapath(r'C:\Users\Big data\Desktop\class\funcardproject\word2vec_model\news\測試100win20cbow1.bin'), binary=True) 

In [17]:
#測試開始_餘閒相似度
input_words= input()
input_vector_matrix =請輸入文字(input_words)
餘閒相似度找文章(input_words)

安安這是一支筆
['安安', '這是', '一支', '筆']

這幾個字的平均向量是:
[[ 0.06511837  0.09959993  0.01864772 -0.06960969 -0.243349   -0.2526291
  -0.03446624 -0.03591082  0.03554634 -0.20151824 -0.02446517  0.3225115
  -0.25768012 -0.2134781  -0.09035534 -0.23705901  0.04721848  0.14462538
  -0.3008927  -0.09443245  0.16557139 -0.1323007  -0.08469781  0.13489021
  -0.70736045  0.30326876  0.21777336 -0.09950419  0.38934323 -0.16862933
  -0.13526939 -0.0278507   0.21139452 -0.17017642 -0.40795097  0.01381686
  -0.02290034 -0.07928631 -0.06565227  0.07718517  0.09386844  0.00094105
  -0.50264806 -0.08453703  0.07512541  0.09366845 -0.00537157 -0.02502326
   0.03734794  0.02389946 -0.17284478  0.03994563 -0.15903616  0.28528208
   0.07108307 -0.09482056 -0.23727123  0.046823   -0.10624644 -0.09052489
  -0.30251288  0.31550232 -0.2611305   0.05618082  0.44768435  0.07490891
   0.08805026 -0.38285413 -0.16141286  0.04710029  0.05145068 -0.05795455
   0.10369752 -0.1060154   0.01954195 -0.29094547  0.20464295  0.2503

In [ ]:
# #4.獲取輸入詞的平均向量
# def get_article_avgvector(wordlist):
#     #取每篇文章平均向量
#     # x=np.matrix(wv_from_bin[word])安安?
# #     len_wordlist = len(wordlist)
#     input_avgvector_matrix=0
#     for word in wordlist:
#         try:
#             x=np.matrix(wv_from_bin[word])
#             input_avgvector_matrix+=x
            
#         except:
#             pass
#     len_wordlist=len(input_avgvector_matrix)
#     input_avgvector_matrix = input_avgvector_matrix/len_wordlist
#     return(input_avgvector_matrix)

In [8]:
#4.獲取輸入詞的平均向量
def get_article_avgvector(wordlist):
    #取每篇文章平均向量
    # x=np.matrix(wv_from_bin[word])安安?
    len_wordlist=0
    input_avgvector_matrix=0
    for word in wordlist:

        try:
            x=np.matrix(wv_from_bin[word])
            input_avgvector_matrix+=x
            len_wordlist+=1

        except:
            pass
    if type(input_avgvector_matrix)==int:
        input_avgvector_matrix = articles_matrix[1]
    else:
        input_avgvector_matrix = input_avgvector_matrix/len_wordlist
    
 
      
    #若沒有輸入文章或輸入的詞不在bin檔則回傳第0篇文章向量

    
     
    return(input_avgvector_matrix)

In [9]:
x=np.matrix(wv_from_bin["旅遊"])
y=np.matrix(wv_from_bin["加油"])
input_avgvector_matrix=0
input_avgvector_matrix+=x
input_avgvector_matrix+=y
input_avgvector_matrix

NameError: name 'wv_from_bin' is not defined

In [10]:
#3.輸入文字
def 請輸入文字(input_words):
    #隨意輸入文字
    #轉DataFrame
    # input_words=pd.DataFrame(input_words,columns=['content'])
    #斷詞
    wordlist=jieba.lcut(input_words,cut_all=False)
    # df1=斷詞(input_words)
    #清乾淨
    # news = 整理(cut)
    # 吐出來
    # output_words = input_words['seg'][0]
    # output_words = output_words.split(' ')
    print(wordlist)
    input_vector_matrix = get_article_avgvector(wordlist)
    print()
    print("這幾個字的平均向量是:")
    print(input_vector_matrix)
    return(input_vector_matrix)

In [11]:
#5.餘弦相似度
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [12]:
#撈出存在excel的向量轉為matrix
def get_article_matrix(article,i):
    aa = article.loc[:,["article_vector_matrix"]][i:i+1]
    #轉ARRAY再轉list
    b = np.array(aa)
    b=b[0].tolist()#list
    # 切
    c = str(b[0]).split(',')
    article_matrix = np.mat(c).astype(float)
    return(article_matrix)

In [13]:
#比對
def 餘閒相似度找文章(input_words):
    articles_matrix_list=[]
    for b in range(5000):
        結果=cos_sim(input_vector_matrix,articles_matrix[b])
        articles_matrix_list.append(結果)
    print("第",articles_matrix_list.index(max(articles_matrix_list)),"篇新聞最相似")
    哭喔 = articles_matrix_list.index(max(articles_matrix_list))
    print("文章內文為:","\n","\n","------------------------------------------------------------","\n",np.array(article[哭喔:哭喔+1]['content'])[0])
    

In [8]:
def 歐式距離找文章(input_words):
    articles_matrix_list=[]
    for b in range(5000):
        結果=np.linalg.norm(input_vector_matrix - articles_matrix[b])
        articles_matrix_list.append(結果)
    
    print("第",articles_matrix_list.index(max(articles_matrix_list)),"篇新聞最相似")
    哭喔 = articles_matrix_list.index(max(articles_matrix_list))
    print("文章內文為:","\n","\n","------------------------------------------------------------","\n",np.array(article[哭喔:哭喔+1]['content'])[0])
     

In [9]:
# 斷詞套件
def 斷詞(news):
    df1=news
    df1['seg']=None
    df1['word_freq']=None
    n=len(news)
    alltext=[]
    stopset=set()
    stop2=['/n','']
    with open('C:\\Users\\Big data\\Desktop\\字詞貼標\\DB104-master\\stop.txt','r',encoding='ISO-8859-1') as s:
        for line in s:
            stopset.add(line.strip('\n'))
    jieba.load_userdict('C:\\Users\\Big data\\Desktop\\字詞貼標\\DB104-master\\userdict.txt')
    for i in range(n):
        seg=''
        wf={}
        text= str(df1.loc[i]['content'])
        cut=jieba.cut(text,cut_all=False)
        for j in cut:
            if j not in stopset:
                seg +=j+' '
                seg = seg.replace("\n",'')
        df1['seg'][i]=seg
        for w in seg.split(' '):
            if w not in wf:
                wf[w]=1
            else:
                wf[w]+=1
        word_list=[(k,wf[k]) for k in wf if k not in stop2 ]
        word_list.sort(key=lambda a :a[1],reverse=True)
        df1['word_freq'][i]=word_list
    return(df1)



# 斷詞整理(去除標點符號)
def 整理(df1):
    datanews = df1["seg"]
    # 轉list
    train_data = np.array(datanews)#np.ndarray()
    datanews=train_data.tolist()#list
    df1=''
    for datanew in datanews:
        df1+=str(datanew)+'\n'
    df1=df1.replace('[','').replace('\'','').replace('「 ','').replace('」 ','').replace('」 ','').replace('、 ','').replace('： ','')
    df1=df1.replace('《 ','').replace('》 ','').replace('） ','').replace('( ','').replace('／ ','').replace('， ','')
    df1=df1.replace('╱ ','').replace('！ ','').replace('？ ','').replace('（ ','').replace('。 ','').replace('； ','').replace('… ','')
    df1=df1.replace(']','').replace(' :','').replace('\u3000','').replace('\n','')
    df2=df1
    for i in range(len(df2)):
        df1=' '.join(df2.split())
    return(df1)